<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/runtime_config/ConfigurableLLM_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Runtime Configurations


We want to configure our graph when calling it. For example, we want to specify LLM or system prompt to use at runtime _without polluting the graph state with these parameters_.

Adding runtime configuration, invloves a) specifying a schema for the configuration, b) adding the configuration to the nodes or conditional edges and c) passing the configuration to the graph

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain langchain_core  langchain_openai langchain-anthropic dotenv

Specify the necessary keys

In [ ]:
import os, getpass
from dotenv import load_dotenv

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

load_dotenv()

_set_env("OPENAI_API_KEY")

_set_env("ANTHROPIC_API_KEY")


## Speciyfing LLM at runtime

Below is a simple example which illustrates how to specify LLM at runtime using the Runtime Configuration functionality:

In [ ]:

from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, StateGraph, START, MessagesState
from typing_extensions import TypedDict

from langchain.chat_models import init_chat_model


1) Specify the context schema

In [ ]:
MODELS = {
    "anthropic": "anthropic:claude-3-5-haiku-latest",
    "openai": "openai:gpt-4.1-mini"
}

class ConfigSchema(TypedDict):
    model_provider: str

def model(state: MessagesState, config: RunnableConfig):
    if "model_provider" in config["configurable"]:
      if config["configurable"]["model_provider"] == "anthropic":
          chat_model = init_chat_model(MODELS["anthropic"])
          response = chat_model.invoke(state["messages"])
          return {"messages": [response]}
      elif config["configurable"]["model_provider"] == "openai":
          chat_model = init_chat_model(MODELS["openai"])
          response = chat_model.invoke(state["messages"])
          return {"messages": [response]}
      else:
          raise ValueError("Unknown values.")
    else:
      # config/model_provider not specified, detault to anthropic
      chat_model = init_chat_model(MODELS["anthropic"])
      response = chat_model.invoke(state["messages"])
      return {"messages": [response]}





In [ ]:
builder = StateGraph(MessagesState, config_schema=ConfigSchema)
builder.add_node(model)
builder.add_edge(START, "model")
builder.add_edge("model", END)

graph = builder.compile()

User the graph with test input message

In [ ]:
input_message = {"role": "user", "content": "hi"}

With no configuration, this graph uses default (Anthropic)

In [ ]:
response_1 = graph.invoke({"messages": [input_message]})["messages"][-1]
print(response_1.response_metadata["model_name"])

claude-3-5-haiku-20241022


with configuration supplied it opens what is specified:

In [ ]:
response_2 = graph.invoke({"messages": [input_message]}, config=ConfigSchema(model_provider="openai"))["messages"][-1]
response_3 = graph.invoke({"messages": [input_message]}, config=ConfigSchema(model_provider="anthropic"))["messages"][-1]

print(response_2.response_metadata["model_name"])
print(response_3.response_metadata["model_name"])

gpt-4.1-mini-2025-04-14
claude-3-5-haiku-20241022
